In [5]:
# 指标stars对应的数据表格式如下：
#  1.  项目1：    "2015-01": 5,
#                 "2015-03": 2,
#                 "2015-08": 2,
#                 "2015-10": 1,
#                      ……
#  2.  项目2：    "2016-01": 6,
#         ……           ……
# 即项目名：年-月-这个月的stars数量
#
# 希望能够将一个项目的stars数量进行汇总，呈现：
#  1.     项目1：    stars总数
#  2.     项目2：    stars总数
#         ……           ……
# 同时筛选出stars数量排名前十的项目


import pymysql
import json


# 连接到 MySQL 数据库
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='cptbtptp789',
    database='project_data',
    charset='utf8mb4'
)

cursor = conn.cursor()

# 删除原表 stars_total_metrics（如果存在）  (这里的处理是由于先前 stars_total_metrics 建立有误)
drop_table_query_stars_total = "DROP TABLE IF EXISTS stars_total_metrics"
try:
    cursor.execute(drop_table_query_stars_total)
    print("原表 stars_total_metrics 已删除（如果存在）")
except pymysql.Error as e:
    print(f"删除原表 stars_total_metrics 时出错: {e}")

# 删除原表 top_10_stars_projects（如果存在）
drop_table_query_top_10 = "DROP TABLE IF EXISTS top_10_stars_projects"
try:
    cursor.execute(drop_table_query_top_10)
    print("原表 top_10_stars_projects 已删除（如果存在）")
except pymysql.Error as e:
    print(f"删除原表 top_10_stars_projects 时出错: {e}")

# 创建新表 stars_total_metrics
create_table_query_stars_total = """
CREATE TABLE IF NOT EXISTS stars_total_metrics (
    id INT AUTO_INCREMENT PRIMARY KEY,
    project_name VARCHAR(255) NOT NULL,
    total_stars INT
)
"""
try:
    cursor.execute(create_table_query_stars_total)
    print("新表 stars_total_metrics 创建成功")
except pymysql.Error as e:
    print(f"创建新表 stars_total_metrics 时出错: {e}")

# 从 stars_metrics 表中查询数据
select_query_stars_total = "SELECT project_name, metric_value FROM stars_metrics"
cursor.execute(select_query_stars_total)

# 获取查询结果
results_stars_total = cursor.fetchall()

# 处理并插入数据到 stars_total_metrics 表
unique_projects = set()
for row in results_stars_total:
    project_name, metric_value_str = row
    if project_name not in unique_projects:
        metric_value = json.loads(metric_value_str)
        total_stars = 0
        for value in metric_value.values():
            if isinstance(value, (int, float)):
                total_stars += value
        insert_query_stars_total = "INSERT INTO stars_total_metrics (project_name, total_stars) VALUES (%s, %s)"
        try:
            cursor.execute(insert_query_stars_total, (project_name, total_stars))
            conn.commit()
            unique_projects.add(project_name)
        except pymysql.Error as e:
            print(f"插入数据到 stars_total_metrics 表时出错: {e}")

# 创建新表 top_10_stars_projects
create_table_query_top_10 = """
CREATE TABLE IF NOT EXISTS top_10_stars_projects (
    id INT AUTO_INCREMENT PRIMARY KEY,
    project_name VARCHAR(255) NOT NULL,
    total_stars INT
)
"""
try:
    cursor.execute(create_table_query_top_10)
    print("新表 top_10_stars_projects 创建成功")
except pymysql.Error as e:
    print(f"创建新表 top_10_stars_projects 时出错: {e}")

# 从 stars_total_metrics 表中筛选出前 10 个收藏数最高的项目
select_query_top_10 = """
SELECT project_name, total_stars
FROM stars_total_metrics
ORDER BY total_stars DESC
LIMIT 10
"""
cursor.execute(select_query_top_10)

# 获取查询结果
results_top_10 = cursor.fetchall()

# 处理并插入数据到 top_10_stars_projects 表
for row in results_top_10:
    project_name, total_stars = row
    insert_query_top_10 = "INSERT INTO top_10_stars_projects (project_name, total_stars) VALUES (%s, %s)"
    try:
        cursor.execute(insert_query_top_10, (project_name, total_stars))
        conn.commit()
    except pymysql.Error as e:
        print(f"插入数据到 top_10_stars_projects 表时出错: {e}")

# 关闭连接
cursor.close()
conn.close()

原表 stars_total_metrics 已删除（如果存在）
原表 top_10_stars_projects 已删除（如果存在）
新表 stars_total_metrics 创建成功
新表 top_10_stars_projects 创建成功
